# 🖊️ SignVerifAI - Optimized Training on A100 GPU

**Features:**
- Batch size: 128
- OneCycleLR scheduler (max_lr=1e-3, min_lr=1e-6)
- Verbose metrics: F1, Precision, Recall, Sensitivity, Confusion Matrix
- Early stopping

**Runtime:** A100 GPU recommended

---

## 1️⃣ GPU Check & Setup

In [ ]:
# GPU kontrolü
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2️⃣ Clone & Install

In [ ]:
# GitHub'dan klonla (en son kod)
!rm -rf SignVerifAI
!git clone https://github.com/gorkemelih/SignVerifAI.git
%cd SignVerifAI

# Bağımlılıkları kur
!pip install -e . -q
print("✅ Kurulum tamamlandı!")

## 3️⃣ Mount Google Drive & Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Drive'dan verileri yükle
import os

DRIVE_DATA_PATH = "/content/drive/MyDrive/SignVerifAI/data_processed.zip"

if os.path.exists(DRIVE_DATA_PATH):
    !unzip -q -o {DRIVE_DATA_PATH} -d .
    print("✅ Veriler yüklendi!")
else:
    print(f"❌ Dosya bulunamadı: {DRIVE_DATA_PATH}")
    print("👉 Lütfen Drive'a data_processed.zip yükleyin.")

## 4️⃣ Prepare Data Splits & Pairs

In [ ]:
# Metadata kontrolü
import pandas as pd

metadata = pd.read_csv('data_processed/metadata.csv')
print(f"Toplam görüntü: {len(metadata)}")
print(f"Kişi sayısı: {metadata['person_id'].nunique()}")
print(f"\nEtiket dağılımı:")
print(metadata['label'].value_counts())

In [ ]:
# Split oluştur
!signverify split

# Pair'leri oluştur (50k train, 10k val)
!signverify pairs

## 5️⃣ Train Model 🚀

**Optimizations:**
- Batch size: 128
- OneCycleLR: max_lr=1e-3, min_lr=1e-6
- Early stopping: patience=10
- Verbose metrics every epoch

In [ ]:
# Eğitim parametreleri
EPOCHS = 100
BATCH_SIZE = 128  # Optimized for A100
LEARNING_RATE = 1e-4  # Initial LR (OneCycleLR will handle)

# Eğitimi başlat
!signverify train --device cuda --epochs {EPOCHS} --batch-size {BATCH_SIZE} --lr {LEARNING_RATE}

## 6️⃣ Evaluate Model

In [ ]:
# Test seti üzerinde değerlendirme
!signverify eval --device cuda

In [ ]:
# Sonuçları görüntüle
from IPython.display import Image, display, Markdown

print("📊 ROC Curve:")
display(Image('outputs/reports/roc_curve.png'))

print("\n📊 Score Distribution:")
display(Image('outputs/reports/score_distribution.png'))

print("\n📊 Confusion Matrix:")
display(Image('outputs/reports/confusion_matrix.png'))

In [ ]:
# Evaluation report
with open('outputs/reports/eval_report.md', 'r') as f:
    display(Markdown(f.read()))

## 7️⃣ Save Model to Drive

In [ ]:
import shutil
import glob

# En son run klasörünü bul
runs = sorted(glob.glob('outputs/models/run_*'))
if runs:
    latest_run = runs[-1]
    dest = "/content/drive/MyDrive/SignVerifAI/trained_models/"
    os.makedirs(dest, exist_ok=True)
    
    # Model dosyasını kopyala
    shutil.copy(f"{latest_run}/checkpoint_best.pt", dest)
    shutil.copy(f"{latest_run}/config.json", dest)
    
    # Reports'u da kopyala
    reports_dest = "/content/drive/MyDrive/SignVerifAI/reports/"
    os.makedirs(reports_dest, exist_ok=True)
    for f in glob.glob('outputs/reports/*'):
        shutil.copy(f, reports_dest)
    
    print(f"✅ Model kaydedildi: {dest}")
    print(f"✅ Raporlar kaydedildi: {reports_dest}")
else:
    print("❌ Eğitilmiş model bulunamadı.")

---
## 📋 Training Summary

| Setting | Value |
|---------|-------|
| Batch Size | 128 |
| Scheduler | OneCycleLR |
| Max LR | 1e-3 |
| Min LR | 1e-6 |
| Early Stopping | Patience 10 |
| Metrics | AUC, EER, F1, Precision, Recall, Sensitivity, Confusion Matrix |